# **Project Description:**

As part of our Communication Program, we are building an AI tool to analyse and score students’ spoken communication skills. One common exercise is a short self-introduction submitted as an audio file. The audio has already been transcribed to text (transcript provided in the Excel file).

Objective of this case study: build a tool (front-end + back-end+ Logics) that takes a transcript text as input and produces a rubric-based final score (0–100) and per-criterion feedback. The tool must combine rule-based methods, NLP-based semantic scoring, and apply the data driven rubric provided in the Excel file.

## **Environment Setup: Installing Libraries**

In [1]:
!pip install streamlit==1.38.0 pandas==2.2.2 numpy==1.26.4 openpyxl==3.1.5 regex==2024.9.11 \
sentence-transformers==3.0.1 scikit-learn==1.4.2 language-tool-python==2.7.1 vaderSentiment==3.3.2

## **Rubric Loader**

In [2]:
# This block defines all rubric criteria directly in Python, based on the Excel and image rubrics.
# Each criterion includes: criterion, description, keywords (if applicable), and weight.

rubric_data = [
    {
        "criterion": "Salutation Level",
        "description": "Quality of greeting at the start of introduction",
        "keywords": ["hi", "hello", "good morning", "good afternoon", "good evening", "good day", "hello everyone", "i am excited to introduce", "feeling great"],
        "weight": 5
    },
    {
        "criterion": "Keyword Presence",
        "description": "Presence of mandatory and optional details: name, age, class, school, family, hobbies, goals, unique point",
        "keywords": [
            # Must-have (4 points each)
            "name", "age", "class", "school", "family", "hobbies", "interest", "free time",
            # Good-to-have (2 points each)
            "about family", "i am from", "parents are from", "ambition", "goal", "dream", "fun fact", "unique", "strength", "achievement"
        ],
        "weight": 30
    },
    {
        "criterion": "Flow",
        "description": "Order followed: Salutation → Basic details → Additional details → Closing",
        "keywords": ["salutation", "basic details", "additional details", "closing"],
        "weight": 5
    },
    {
        "criterion": "Speech Rate",
        "description": "Words per minute (WPM) speed of speaking",
        "keywords": [],
        "weight": 10
    },
    {
        "criterion": "Grammar",
        "description": "Grammar correctness using LanguageTool",
        "keywords": [],
        "weight": 10
    },
    {
        "criterion": "Vocabulary Richness",
        "description": "Lexical diversity measured by TTR (Type-Token Ratio)",
        "keywords": [],
        "weight": 10
    },
    {
        "criterion": "Clarity",
        "description": "Filler word rate (um, uh, like, you know, etc.)",
        "keywords": ["um", "uh", "like", "you know", "so", "actually", "basically", "right", "i mean", "well", "kinda", "sort of", "okay", "hmm", "ah"],
        "weight": 15
    },
    {
        "criterion": "Engagement",
        "description": "Sentiment/positivity of transcript using VADER",
        "keywords": [],
        "weight": 15
    }
]

# Quick check: print all criteria and weights
for r in rubric_data:
    print(f"{r['criterion']:25} → weight: {r['weight']}")


Salutation Level          → weight: 5
Keyword Presence          → weight: 30
Flow                      → weight: 5
Speech Rate               → weight: 10
Grammar                   → weight: 10
Vocabulary Richness       → weight: 10
Clarity                   → weight: 15
Engagement                → weight: 15


## **Transcript Processing**

In [3]:
# This block cleans the transcript, counts words and sentences, and detects filler words.

import re

# Define filler words from rubric
FILLER_WORDS = set([
    "um", "uh", "like", "you know", "so", "actually", "basically", "right",
    "i mean", "well", "kinda", "sort of", "okay", "hmm", "ah"
])

def preprocess_transcript(text: str):

    # Normalize text
    cleaned = text.lower().strip()
    cleaned = re.sub(r"\s+", " ", cleaned)  # collapse whitespace
    cleaned = re.sub(r"[^\w\s]", "", cleaned)  # remove punctuation

    # Word and sentence counts
    words = cleaned.split()
    word_count = len(words)
    sentence_count = text.count(".") + text.count("!") + text.count("?")  # rough estimate

    # Filler word detection
    filler_hits = [w for w in words if w in FILLER_WORDS]
    filler_count = len(filler_hits)
    filler_rate = round((filler_count / word_count) * 100, 2) if word_count > 0 else 0

    return {
        "cleaned_text": cleaned,
        "word_count": word_count,
        "sentence_count": sentence_count,
        "filler_count": filler_count,
        "filler_rate": filler_rate,
        "filler_words_found": filler_hits
    }

sample_text = """Hello everyone, myself Muskan, studying in class 8th B section from Christ Public School.
I am 13 years old. I live with my family. There are 3 people in my family, me, my mother and my father.
One special thing about my family is that they are very kind hearted to everyone and soft spoken. One thing I really enjoy is play, playing cricket and taking wickets.
A fun fact about me is that I see in mirror and talk by myself. One thing people don't know about me is that I once stole a toy from one of my cousin.
My favorite subject is science because it is very interesting. Through science I can explore the whole world and make the discoveries and improve the lives of others.
Thank you for listening."""

preprocessed = preprocess_transcript(sample_text)
print(preprocessed)

{'cleaned_text': 'hello everyone myself muskan studying in class 8th b section from christ public school i am 13 years old i live with my family there are 3 people in my family me my mother and my father one special thing about my family is that they are very kind hearted to everyone and soft spoken one thing i really enjoy is play playing cricket and taking wickets a fun fact about me is that i see in mirror and talk by myself one thing people dont know about me is that i once stole a toy from one of my cousin my favorite subject is science because it is very interesting through science i can explore the whole world and make the discoveries and improve the lives of others thank you for listening', 'word_count': 133, 'sentence_count': 11, 'filler_count': 0, 'filler_rate': 0.0, 'filler_words_found': []}


## **Salutation and Keyword Scoring**

In [4]:
# This block scores two rule-based criteria:
# - Salutation Level: based on greeting phrases
# - Keyword Presence: checks for must-have and good-to-have keywords

def score_salutation(text: str):
    """
    Score salutation level based on presence of greeting phrases.
    Returns score (0–5) and matched phrase.
    """
    text_lower = text.lower()
    if "i am excited to introduce" in text_lower or "feeling great" in text_lower:
        return 5, "Excellent"
    elif any(phrase in text_lower for phrase in ["good morning", "good afternoon", "good evening", "good day", "hello everyone"]):
        return 4, "Good"
    elif any(phrase in text_lower for phrase in ["hi", "hello"]):
        return 2, "Normal"
    else:
        return 0, "No Salutation"

def score_keywords(text: str):
    """
    Score keyword presence based on must-have and good-to-have keywords.
    Returns score (0–30), list of matched keywords, and missing ones.
    """
    text_lower = text.lower()
    words = set(text_lower.split())

    # Define must-have (4 pts each) and good-to-have (2 pts each)
    must_have = ["name", "age", "class", "school", "family", "hobbies", "interest", "free time"]
    good_to_have = ["about family", "i am from", "parents are from", "ambition", "goal", "dream", "fun fact", "unique", "strength", "achievement"]

    score = 0
    matched = []
    missing = []

    # Check must-have
    for kw in must_have:
        if kw in text_lower:
            score += 4
            matched.append(kw)
        else:
            missing.append(kw)

    # Check good-to-have
    for kw in good_to_have:
        if kw in text_lower:
            score += 2
            matched.append(kw)
        else:
            missing.append(kw)

    # Cap score at 30
    score = min(score, 30)

    return score, matched, missing

# Example usage:
sal_score, sal_type = score_salutation(sample_text)
kw_score, kw_matched, kw_missing = score_keywords(sample_text)

print(f"Salutation Score: {sal_score} ({sal_type})")
print(f"Keyword Score: {kw_score}")
print(f"Matched Keywords: {kw_matched}")
print(f"Missing Keywords: {kw_missing}")

Salutation Score: 4 (Good)
Keyword Score: 18
Matched Keywords: ['class', 'school', 'family', 'interest', 'fun fact']
Missing Keywords: ['name', 'age', 'hobbies', 'free time', 'about family', 'i am from', 'parents are from', 'ambition', 'goal', 'dream', 'unique', 'strength', 'achievement']


## **Flow Scoring**

In [5]:
# This block checks whether the transcript follows the expected order:
# Salutation → Basic Details → Additional Details → Closing

def score_flow(text: str):
    """
    Score flow based on presence and order of key sections.
    Returns score (0 or 5) and detected order.
    """
    text_lower = text.lower()

    # Define section anchors
    salutation_phrases = ["hello", "hi", "good morning", "good afternoon", "good evening", "i am excited"]
    basic_details = ["my name is", "i am", "class", "school", "age"]
    additional_details = ["hobbies", "interest", "fun fact", "goal", "dream", "unique", "strength", "achievement"]
    closing_phrases = ["thank you", "thanks for listening", "that's all"]

    # Find positions of each section
    def find_first(text, phrases):
        for phrase in phrases:
            idx = text.find(phrase)
            if idx != -1:
                return idx
        return -1

    sal_idx = find_first(text_lower, salutation_phrases)
    basic_idx = find_first(text_lower, basic_details)
    add_idx = find_first(text_lower, additional_details)
    close_idx = find_first(text_lower, closing_phrases)

    # Check if order is followed
    positions = [sal_idx, basic_idx, add_idx, close_idx]
    valid_positions = [p for p in positions if p != -1]

    if valid_positions == sorted(valid_positions):
        return 5, "Order followed"
    else:
        return 0, "Order not followed"

# Example usage:
flow_score, flow_feedback = score_flow(sample_text)
print(f"Flow Score: {flow_score} ({flow_feedback})")

Flow Score: 5 (Order followed)


## **Speech Rate Scoring**

In [6]:
# This block calculates words per minute (WPM) and assigns a score based on rubric thresholds.

def score_speech_rate(word_count: int, duration_sec: int):

    if duration_sec <= 0:
        return 0, 0, "Invalid duration"

    wpm = (word_count / duration_sec) * 60  # words per minute

    # Apply rubric thresholds
    if wpm > 161:
        return 2, round(wpm, 2), "Too Fast"
    elif 141 <= wpm <= 160:
        return 6, round(wpm, 2), "Fast"
    elif 111 <= wpm <= 140:
        return 10, round(wpm, 2), "Ideal"
    elif 81 <= wpm <= 110:
        return 6, round(wpm, 2), "Slow"
    elif wpm < 80:
        return 2, round(wpm, 2), "Too Slow"
    else:
        return 0, round(wpm, 2), "Unscored"

# Example usage:
duration_sec = 52  # from rubric sample
speech_score, wpm_value, wpm_category = score_speech_rate(preprocessed["word_count"], duration_sec)

print(f"Speech Rate Score: {speech_score} ({wpm_category}, {wpm_value} WPM)")

Speech Rate Score: 6 (Fast, 153.46 WPM)


## **Grammar Scoring**

In [8]:
# - Detect repeated words
# - Detect missing capitalization at sentence start
# - Detect very long sentences (run-ons)

import re

def score_grammar_simple(text: str):

    words = text.split()
    word_count = len(words)

    # Heuristic 1: repeated words
    repeated_errors = sum(1 for i in range(1, len(words)) if words[i].lower() == words[i-1].lower())

    # Heuristic 2: sentences not starting with capital letter
    sentences = re.split(r'[.!?]', text)
    capitalization_errors = sum(1 for s in sentences if s.strip() and not s.strip()[0].isupper())

    # Heuristic 3: very long sentences (>30 words)
    long_sentence_errors = sum(1 for s in sentences if len(s.split()) > 30)

    error_count = repeated_errors + capitalization_errors + long_sentence_errors

    # Errors per 100 words
    errors_per_100 = (error_count / word_count) * 100 if word_count > 0 else 0

    # Grammar score formula (same as rubric)
    grammar_score_ratio = 1 - min(errors_per_100 / 10, 1)

    # Map to rubric bands
    if grammar_score_ratio > 0.9:
        score = 10
    elif 0.7 <= grammar_score_ratio <= 0.89:
        score = 8
    elif 0.5 <= grammar_score_ratio <= 0.69:
        score = 6
    elif 0.3 <= grammar_score_ratio <= 0.49:
        score = 4
    else:
        score = 2

    return score, error_count, grammar_score_ratio

grammar_score, grammar_errors, grammar_ratio = score_grammar_simple(sample_text)
print(f"Grammar Score: {grammar_score} (Errors: {grammar_errors}, Ratio: {grammar_ratio:.2f})")

Grammar Score: 10 (Errors: 0, Ratio: 1.00)


## **Vocabulary Richness Scoring**

In [9]:
# This block calculates lexical diversity using Type-Token Ratio (TTR).

def score_vocabulary(text: str):
    """
    Calculate vocabulary richness using TTR (Type-Token Ratio).
    Returns score (2–10), TTR value, and category.
    """
    words = text.lower().split()
    total_words = len(words)
    distinct_words = len(set(words))

    if total_words == 0:
        return 0, 0, "No words"

    ttr = distinct_words / total_words

    if 0.9 <= ttr <= 1.0:
        score = 10
        category = "Excellent"
    elif 0.7 <= ttr <= 0.89:
        score = 8
        category = "Good"
    elif 0.5 <= ttr <= 0.69:
        score = 6
        category = "Average"
    elif 0.3 <= ttr <= 0.49:
        score = 4
        category = "Poor"
    else:  # 0–0.29
        score = 2
        category = "Very Poor"

    return score, round(ttr, 2), category

vocab_score, vocab_ttr, vocab_category = score_vocabulary(sample_text)
print(f"Vocabulary Score: {vocab_score} ({vocab_category}, TTR={vocab_ttr})")

Vocabulary Score: 6 (Average, TTR=0.68)


## **Clarity Scoring**

In [10]:
# This block calculates filler word rate and assigns a score based on rubric thresholds.

def score_clarity(preprocessed: dict):
    """
    Calculate clarity score based on filler word rate.
    Returns score (3–15), filler count, filler rate, and category.
    """
    filler_count = preprocessed["filler_count"]
    filler_rate = preprocessed["filler_rate"]

    # Apply rubric thresholds
    if filler_rate <= 3:
        score = 15
        category = "Excellent"
    elif 4 <= filler_rate <= 6:
        score = 12
        category = "Good"
    elif 7 <= filler_rate <= 9:
        score = 9
        category = "Average"
    elif 10 <= filler_rate <= 12:
        score = 6
        category = "Poor"
    else:  # 13% and above
        score = 3
        category = "Very Poor"

    return score, filler_count, filler_rate, category

clarity_score, filler_count, filler_rate, clarity_category = score_clarity(preprocessed)
print(f"Clarity Score: {clarity_score} ({clarity_category}, {filler_count} fillers, {filler_rate}% rate)")

Clarity Score: 15 (Excellent, 0 fillers, 0.0% rate)


## **Engagement Scoring using VADER**

In [11]:
# This block uses VADER sentiment analysis to measure positivity and assign engagement score.

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def score_engagement(text: str):
    """
    Calculate engagement score using VADER sentiment positivity.
    Returns score (3–15), positivity value, and category.
    """
    sentiment = analyzer.polarity_scores(text)
    positivity = sentiment["pos"]  # probability of positive sentiment (0–1)

    if positivity >= 0.9:
        score = 15
        category = "Highly Positive"
    elif 0.7 <= positivity <= 0.89:
        score = 12
        category = "Positive"
    elif 0.5 <= positivity <= 0.69:
        score = 9
        category = "Neutral/Moderate"
    elif 0.3 <= positivity <= 0.49:
        score = 6
        category = "Low Positive"
    else:  # <0.3
        score = 3
        category = "Negative/Disengaged"

    return score, positivity, category

engagement_score, positivity_value, engagement_category = score_engagement(sample_text)
print(f"Engagement Score: {engagement_score} ({engagement_category}, Positivity={positivity_value:.2f})")

Engagement Score: 3 (Negative/Disengaged, Positivity=0.19)


## **Final Score**

In [12]:
# This block aggregates all rubric scores into a weighted total and prints a detailed breakdown.

def evaluate_transcript(text: str, duration_sec: int):

    # Preprocess
    prep = preprocess_transcript(text)

    sal_score, sal_type = score_salutation(text)
    kw_score, kw_matched, kw_missing = score_keywords(text)
    flow_score, flow_feedback = score_flow(text)
    speech_score, wpm_value, wpm_category = score_speech_rate(prep["word_count"], duration_sec)
    grammar_score, grammar_errors, grammar_ratio = score_grammar_simple(text)
    vocab_score, vocab_ttr, vocab_category = score_vocabulary(text)
    clarity_score, filler_count, filler_rate, clarity_category = score_clarity(prep)
    engagement_score, positivity_value, engagement_category = score_engagement(text)

    # Weighted total (weights from rubric_data)
    total_score = (
        sal_score +
        kw_score +
        flow_score +
        speech_score +
        grammar_score +
        vocab_score +
        clarity_score +
        engagement_score
    )

    # Build report
    report = {
        "Salutation": {"score": sal_score, "type": sal_type},
        "Keywords": {"score": kw_score, "matched": kw_matched, "missing": kw_missing},
        "Flow": {"score": flow_score, "feedback": flow_feedback},
        "Speech Rate": {"score": speech_score, "wpm": wpm_value, "category": wpm_category},
        "Grammar": {"score": grammar_score, "errors": grammar_errors, "ratio": round(grammar_ratio, 2)},
        "Vocabulary": {"score": vocab_score, "ttr": vocab_ttr, "category": vocab_category},
        "Clarity": {"score": clarity_score, "fillers": filler_count, "rate": filler_rate, "category": clarity_category},
        "Engagement": {"score": engagement_score, "positivity": round(positivity_value, 2), "category": engagement_category},
        "Total Score": total_score
    }

    return report

final_report = evaluate_transcript(sample_text, duration_sec=52)
for criterion, details in final_report.items():
    print(f"{criterion}: {details}")

Salutation: {'score': 4, 'type': 'Good'}
Keywords: {'score': 18, 'matched': ['class', 'school', 'family', 'interest', 'fun fact'], 'missing': ['name', 'age', 'hobbies', 'free time', 'about family', 'i am from', 'parents are from', 'ambition', 'goal', 'dream', 'unique', 'strength', 'achievement']}
Flow: {'score': 5, 'feedback': 'Order followed'}
Speech Rate: {'score': 6, 'wpm': 153.46, 'category': 'Fast'}
Grammar: {'score': 10, 'errors': 0, 'ratio': 1.0}
Vocabulary: {'score': 6, 'ttr': 0.68, 'category': 'Average'}
Clarity: {'score': 15, 'fillers': 0, 'rate': 0.0, 'category': 'Excellent'}
Engagement: {'score': 3, 'positivity': 0.19, 'category': 'Negative/Disengaged'}
Total Score: 67


## **Streamlit Deployement**

In [16]:
%%writefile app.py

import streamlit as st

def run_app():
    st.title("Communication Scoring Tool")
    st.write("Enter your introduction transcript below and get a detailed rubric-based score.")

    # User input text area
    user_text = st.text_area("Type your transcript here:", height=200)

    # Duration input (seconds)
    duration_sec = st.number_input("Enter speech duration (in seconds):", min_value=1, value=60)

    if st.button("Evaluate Transcript"):
        if user_text.strip():

            report = evaluate_transcript(user_text, duration_sec)

            st.subheader("📊 Scoring Breakdown")
            for criterion, details in report.items():
                st.write(f"**{criterion}**: {details}")

            st.success(f"✅ Final Total Score: {report['Total Score']}")
        else:
            st.warning("Please enter a transcript before evaluating.")

if __name__ == "__main__":
    run_app()

Overwriting app.py
